# Binary encoding

In [101]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, BatchNormalization, MaxPooling2D, Flatten, Dense, Dropout, DepthwiseConv2D
import tensorflow as tf

In [102]:
# Función para convertir binario a Gray code
def binary_to_gray(binary_str):
    return binary_str[0] + ''.join(str(int(binary_str[i-1]) ^ int(binary_str[i])) for i in range(1, len(binary_str)))

# Función para convertir Gray code a binario
def gray_to_binary(gray_str):
    binary_str = gray_str[0]
    for i in range(1, len(gray_str)):
        binary_str += str(int(binary_str[i - 1]) ^ int(gray_str[i]))
    return binary_str

# Diccionarios para codificación/decodificación
layer_type_options = {
    'Conv2D': '000', 
    'BatchNorm': '001', 
    'MaxPooling': '010', 
    'Dropout': '011', 
    'Dense': '100', 
    'Flatten': '101',
    'DepthwiseConv2D': '110',  # Mantenemos DepthwiseConv2D
    'DontCare': '111'  # Capa "don't care"
}
filter_options = {32: '00', 30: '01', 16: '10', 8: '11'}
stride_options = {1: '0', 2: '1'}
dropout_options = {0.2: '00', 0.3: '01', 0.4: '10', 0.5: '11'}
neuron_options = {256: '00', 128: '01', 32: '10', 1: '11'}
activation_options = {'relu': '00', 'leaky_relu': '01', 'sigmoid': '10', 'tanh': '11'}

# Función para codificar parámetros de las capas
def encode_layer_params(layer_type, filters=None, strides=None, dropout=None, neurons=None, activation=None):
    binary_representation = layer_type_options.get(layer_type, '000')
    
    if layer_type == 'Conv2D':
        binary_representation += filter_options.get(filters, '00')
        binary_representation += stride_options.get(strides, '0')
        binary_representation += activation_options.get(activation, '00')
    
    if layer_type == 'DepthwiseConv2D':
        binary_representation += filter_options.get(filters, '00')
        binary_representation += stride_options.get(strides, '0')
        binary_representation += activation_options.get(activation, '00')
      
    
    elif layer_type == 'GlobalAveragePooling2D':
        return layer_type_options['GlobalAveragePooling2D']
    
    elif layer_type == 'MaxPooling':
        binary_representation += stride_options.get(strides, '0')
    
    elif layer_type == 'Dropout':
        binary_representation += dropout_options.get(dropout, '00')
    
    elif layer_type == 'Dense':
        binary_representation += neuron_options.get(neurons, '00')
        binary_representation += activation_options.get(activation, '00')
    
     # Fill remaining bits to reach 8 bits
    binary_representation = binary_representation.ljust(8, '0')
    
    return binary_representation

# Función para decodificar parámetros de las capas
def decode_layer_params(encoded_binary):
    # Los primeros 3 bits indican el tipo de capa
    layer_type = encoded_binary[:3]
    
    # Decodificar Conv2D
    if layer_type == '000':  # Conv2D
        filters = decode_value(encoded_binary[3:5], filter_options, 32)
        strides = decode_value(encoded_binary[5], stride_options, 1)
        activation = decode_value(encoded_binary[6:8], activation_options, 'relu')
        return {
            'type': 'Conv2D',
            'filters': filters,
            'kernel_size': (3, 3),  # Constante
            'strides': strides,
            'padding': 'same',  # Constante
            'activation': activation
        }
    elif layer_type == '001':  # BatchNorm
        return {'type': 'BatchNorm'}
   
    
    # Decodificar MaxPooling
    elif layer_type == '010':  # MaxPooling
        strides = decode_value(encoded_binary[3], stride_options, 1)
        return {'type': 'MaxPooling', 'strides': strides}
    
    # Decodificar Dropout
    elif layer_type == '011':  # Dropout
        rate = decode_value(encoded_binary[3:5], dropout_options, 0.2)
        return {'type': 'Dropout', 'rate': rate}
    
    # Decodificar Dense
    elif layer_type == '100':  # Dense
        neurons = decode_value(encoded_binary[3:5], neuron_options, 256)
        activation = decode_value(encoded_binary[5:7], activation_options, 'relu')
        return {'type': 'Dense', 'units': neurons, 'activation': activation}
   
    elif layer_type == '101':  # Flatten
        return {'type': 'Flatten'}
    
     
    # Decodificar DepthwiseConv2D
    elif layer_type == '110':  # DepthwiseConv2D
        filters = decode_value(encoded_binary[3:5], filter_options, 32)
        strides = decode_value(encoded_binary[5], stride_options, 1)
        activation = decode_value(encoded_binary[6:8], activation_options, 'relu')
        return {
            'type': 'DepthwiseConv2D',
            'filters': filters,
            'kernel_size': (3, 3),  # Constante
            'strides': strides,
            'padding': 'same',  # Constante
            'activation': activation
        }
        
    # Caso de "don't care"
    elif layer_type == '111':  # Don't care (relleno)
        return {'type': "DontCare"}  # Se omite
    
    # Si es un tipo no especificado, regresamos None
    else:
        return None

# Función auxiliar para decodificar valores en función de los diccionarios
def decode_value(bits, options_dict, default_value):
    # Buscar los bits en los valores del diccionario y devolver la clave correspondiente
    for key, value in options_dict.items():
        if value == bits:
            return key
    # Si no se encuentra, devolver el valor por defecto
    return default_value



# Ejemplo de uso optimizado
binary_conv2d = encode_layer_params('Conv2D', filters=32, strides=1, activation='relu')
decoded_conv2d = decode_layer_params(binary_conv2d)
print(f"\nBinary encoding of Conv2D: {binary_conv2d}")
print(f"Decoded Conv2D: {decoded_conv2d}")

binary_dropout = encode_layer_params('Dropout', dropout=0.3)
decoded_dropout = decode_layer_params(binary_dropout)
print(f"\nBinary encoding of Dropout: {binary_dropout}")
print(f"Decoded Dropout: {decoded_dropout}")

binary_dense = encode_layer_params('Dense', neurons=128, activation='relu')
decoded_dense = decode_layer_params(binary_dense)
print(f"\nBinary encoding of Dense: {binary_dense}")
print(f"Decoded Dense: {decoded_dense}")


binary_dense = encode_layer_params('DontCare')
decoded_dense = decode_layer_params(binary_dense)
print(f"\nBinary encoding of Dense: {binary_dense}")
print(f"Decoded Dense: {decoded_dense}")



Binary encoding of Conv2D: 00000000
Decoded Conv2D: {'type': 'Conv2D', 'filters': 32, 'kernel_size': (3, 3), 'strides': 1, 'padding': 'same', 'activation': 'relu'}

Binary encoding of Dropout: 01101000
Decoded Dropout: {'type': 'Dropout', 'rate': 0.3}

Binary encoding of Dense: 10001000
Decoded Dense: {'type': 'Dense', 'units': 128, 'activation': 'relu'}

Binary encoding of Dense: 11100000
Decoded Dense: {'type': 'DontCare'}


# Binary Encoding of Building Blocks for Neural Network Architectures

This module implements the **binary encoding** of various building blocks that make up the architecture of a neural network. Each building block (layers like Conv2D, MaxPooling, Dense, etc.) is represented in binary format based on its key parameters (number of filters, kernel size, number of neurons, activations, etc.). This encoding is useful for optimization processes like **NSGA-III** and later conversion to **Gray code**.

### Function `encode_layer_params`

This function generates the binary representation of the parameters of a given layer, depending on the type of layer and its specific parameters.

### Function Parameters

- `layer_type`: (str) The type of layer. It can be one of the following:
  - `Conv2D`
  - `BatchNorm`
  - `MaxPooling`
  - `Dropout`
  - `Dense`
  - `Flatten`
  
- `filters`: (int) [Optional] Number of filters for `Conv2D` layers. Valid options: `32`, `30`, `16`, `8`.

- `kernel_size`: (tuple) [Optional] Kernel size for `Conv2D`. Currently, only `(3, 3)` is supported.

- `strides`: (int) [Optional] Stride used in `Conv2D` or `MaxPooling`. Valid options: `1` and `2`.

- `padding`: (str) [Optional] Padding type in `Conv2D`. Currently, only `'same'` is supported.

- `dropout`: (float) [Optional] Dropout rate for `Dropout` layers. Valid options: `0.2`, `0.3`, `0.5`.

- `neurons`: (int) [Optional] Number of neurons in `Dense` layers. Valid options: `256`, `128`, `32`.

- `activation`: (str) [Optional] Activation function for `Conv2D` or `Dense` layers. Valid options: `'ReLU'`, `'LeakyReLU'`, `'Sigmoid'`.

### Return

- `binary_representation`: (str) A string of bits representing the layer parameters in binary format.

## Layer Type Encoding

Each layer type is represented by a specific sequence of bits. The layer types and their corresponding encodings are:

- **Conv2D**: `'000'`
- **BatchNorm**: `'001'`
- **MaxPooling**: `'010'`
- **Dropout**: `'011'`
- **Dense**: `'100'`
- **Flatten**: `'101'`

## Parameters Encoded by Layer Type

### Conv2D
The `Conv2D` layer includes the following encoded parameters:

- **Number of filters**:
  - `32`: `'00'`
  - `30`: `'01'`
  - `16`: `'10'`
  - `8`: `'11'`

- **Kernel size**:
  - `(3, 3)`: `'0'` (only this size is supported for now)

- **Stride**:
  - `1`: `'0'`
  - `2`: `'1'`

- **Padding**:
  - `'same'`: `'0'`

- **Activation function**:
  - `ReLU`: `'00'`
  - `LeakyReLU`: `'01'`

### BatchNorm
The `BatchNorm` layer has no additional parameters, only its layer type encoding (`'001'`).

### MaxPooling
The `MaxPooling` layer has a single encoded parameter:

- **Stride**:
  - `1`: `'0'`
  - `2`: `'1'`

### Dropout
The `Dropout` layer has the following encoded dropout rates:

- **Dropout Rate**:
  - `0.2`: `'00'`
  - `0.3`: `'01'`
  - `0.5`: `'10'`

### Dense
The `Dense` layer has the following encoded parameters:

- **Number of neurons**:
  - `256`: `'00'`
  - `128`: `'01'`
  - `32`: `'10'`

- **Activation function**:
  - `ReLU`: `'00'`
  - `LeakyReLU`: `'01'`
  - `Sigmoid`: `'10'`

### Flatten
The `Flatten` layer has no additional parameters, only its layer type encoding (`'101'`).

## Gray Code Conversion

In addition to binary encoding, this module supports **Gray code** conversion for efficient optimization, where only one bit changes between consecutive values.

### Function `binary_to_gray`

This function converts a binary string to Gray code using the following logic:
- The first bit in Gray code is the same as the first bit in binary.
- For each subsequent bit, the Gray code bit is the XOR of the current binary bit and the previous binary bit.

### Function Parameters

- `binary_str`: (str) A string representing a binary number.

### Return

- `gray_str`: (str) A string representing the corresponding Gray code.

### Function `gray_to_binary`

This function converts a Gray code string back to its binary equivalent using the following logic:
- The first bit of the binary string is the same as the first bit of the Gray code.
- For each subsequent bit, the binary bit is the XOR of the previous binary bit and the current Gray code bit.

### Function Parameters

- `gray_str`: (str) A string representing a Gray code number.

### Return

- `binary_str`: (str) A string representing the corresponding binary number.




In [109]:
# Función para eliminar campos adicionales como 'padding' y 'kernel_size'
def clean_decoded_model(model_dict):
    cleaned_layers = []
    
    # Recorrer cada capa del modelo decodificado
    for layer in model_dict['layers']:
        # Eliminar las capas 'DontCare'
        if layer['type'] == 'DontCare':
            continue  # No incluir la capa 'DontCare'
        
        # Remover 'kernel_size' y 'padding' si están presentes
        if 'kernel_size' in layer:
            del layer['kernel_size']
        if 'padding' in layer:
            del layer['padding']
        
        # Añadir la capa limpia a la lista
        cleaned_layers.append(layer)
    
    # Devolver el modelo con las capas limpias
    return {'layers': cleaned_layers}


def fixArch(model_dict):
    """
    Corrige y valida la arquitectura del modelo, asegurando que el orden de las capas
    sea coherente y las capas Flatten no aparezcan antes de capas convolucionales o MaxPooling.
    También se aseguran las capas Flatten antes de Dense.
    """
    fixed_layers = []
    input_is_flattened = False  # Bandera para verificar si las entradas están aplanadas
    conv_or_pool_found = False  # Bandera para verificar si encontramos capas Conv o MaxPooling

    for layer in model_dict['layers']:
        # Si encontramos una capa Flatten después de Conv2D, DepthwiseConv2D o MaxPooling
        if layer['type'] == 'Flatten':
            # Si ya hemos encontrado una capa Conv o MaxPooling, es válido tener Flatten
            if conv_or_pool_found:
                input_is_flattened = True
            else:
                # Si no hay capas Conv antes, reemplazar Flatten con DontCare
                print(f"Invalid placement: Replacing Flatten with DontCare.")
                fixed_layers.append({'type': 'DontCare'})
                continue  # Saltar la capa actual
        
        # Si encontramos una capa Conv2D o DepthwiseConv2D después de una capa Flatten
        if input_is_flattened and layer['type'] in ['Conv2D', 'DepthwiseConv2D', 'MaxPooling']:
            print(f"Invalid placement: Replacing {layer['type']} with DontCare after Flatten.")
            layer = {'type': 'DontCare'}
        
        # Si encontramos una capa Dense sin haber pasado por Flatten
        if layer['type'] == 'Dense' and not input_is_flattened:
            print(f"Inserting Flatten before Dense layer.")
            fixed_layers.append({'type': 'Flatten'})
            input_is_flattened = True  # Ahora las entradas están aplanadas
        
        # Si encontramos capas Conv2D o MaxPooling, activamos el flag
        if layer['type'] in ['Conv2D', 'DepthwiseConv2D', 'MaxPooling']:
            conv_or_pool_found = True

        # Agregar la capa (ya corregida) a la lista de capas fijas
        fixed_layers.append(layer)

    # Retorna el modelo con las capas corregidas
    model_dict['layers'] = fixed_layers
    return model_dict




def decode_model_architecture(encoded_string_binary, gene_length=8):
    model_dict = {'layers': []}
    
    index = 0
    while index < len(encoded_string_binary):
        # Extraer los próximos 8 bits de la codificación binaria
        binary_code_for_layer = encoded_string_binary[index:index+gene_length]
        
        # Decodificar una capa completa usando decode_layer_params
        decoded_layer = decode_layer_params(binary_code_for_layer)
        
        # Añadir la capa decodificada al modelo
        model_dict['layers'].append(decoded_layer)
        
        # Avanzar el índice por el tamaño del gen (8 bits)
        index += gene_length
    
    return fixArch(model_dict)




# Función para codificar la arquitectura del modelo con relleno hasta 75 alelos
def encode_model_architecture(model_dict, max_alleles=104):
    encoded_layers_bin = []
    total_alleles = 0
    
    # Codificar cada capa individualmente
    for layer in model_dict['layers']:
        binary_encoding = encode_layer_params(
            layer_type=layer['type'],
            filters=layer.get('filters'),
            strides=layer.get('strides'),
            dropout=layer.get('rate'),
            neurons=layer.get('units'),
            activation=layer.get('activation', '').lower()
        )
        
        # Contar el número de alelos actuales
        total_alleles += len(binary_encoding)
        encoded_layers_bin.append(binary_encoding)
        print(f"Layer: {layer} -> Binary Encoding: {binary_encoding}")
    print(f"Total Alleles: {total_alleles}")
    # Verificar si se necesitan capas DontCare
    while total_alleles < max_alleles:
        # Codificar una capa DontCare (relleno)
        dont_care_encoding = encode_layer_params('DontCare')
        encoded_layers_bin.append(dont_care_encoding)
        
        # Aumentar el número de alelos
        total_alleles += len(dont_care_encoding)
        #print(f"Added DontCare Layer -> Binary Encoding: {dont_care_encoding}")
    print(f"Total Alleles (after padding): {total_alleles}")
    # Concatenar todas las codificaciones en una sola cadena
    final_encoding_bin = ''.join(encoded_layers_bin[:max_alleles])  # Asegurarse de no exceder el número máximo de alelos
    print(f"Final Binary Encoding: {final_encoding_bin}")

    return final_encoding_bin

# Función de verificación
def verify_model_architecture(original_model):
    # Codificar el modelo original
    encoded_model = encode_model_architecture(original_model)
    
    # Decodificar el modelo desde el encoding binario
    decoded_model = decode_model_architecture(encoded_model)
    print("\nDecoded Model:")
    print(decoded_model)
    fixed_model = fixArch(decoded_model)
    print("\nFixed Model:")
    print(fixed_model)
    # Limpiar el modelo decodificado para quitar 'kernel_size', 'padding' y capas DontCare
    decoded_model_cleaned = clean_decoded_model(fixed_model)
    
    # Verificar si el modelo original y decodificado son iguales
    if original_model == decoded_model_cleaned:
        print("Modelo original y decodificado son iguales.")
        return True
    else:
        print("Original Model:")
        print(original_model)
        print("\nDecoded Model (cleaned):")
        print(decoded_model_cleaned)
        return False

# Ejemplo de uso
model_example = {
    "layers": [
        {"type": "Flatten"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "relu"},
        {"type": "BatchNorm"},
        {"type": "MaxPooling", "strides": 2},
        {"type": "Flatten"},
        {"type": "Dense", "units": 256, "activation": "relu"},
        {"type": "Dropout", "rate": 0.5},
        {"type": "Dense", "units": 1, "activation": "sigmoid"}
    ]
}

# Verificar el modelo
print(verify_model_architecture(model_example))

# Construcción del modelo en TensorFlow desde el diccionario decodificado

class DontCareLayer(tf.keras.layers.Layer):
    def __init__(self):
        super(DontCareLayer, self).__init__()

    def call(self, inputs):
        # Devuelve los inputs sin ninguna modificación (identidad)
        return inputs

# Construcción del modelo en TensorFlow desde el diccionario decodificado
def build_tf_model_from_dict(model_dict, input_shape=(28, 28, 3)):
    model = Sequential()
    
    for layer in model_dict['layers']:
        if layer['type'] == 'Conv2D':
            model.add(Conv2D(filters=layer['filters'], 
                             kernel_size=(3, 3), 
                             strides=layer['strides'], 
                             padding='same', 
                             activation=layer['activation']))
                             
        elif layer['type'] == 'DepthwiseConv2D':
            model.add(DepthwiseConv2D(kernel_size=(3, 3), 
                                      strides=layer['strides'], 
                                      padding='same', 
                                      activation=layer['activation']))

        elif layer['type'] == 'BatchNorm':
            model.add(BatchNormalization())
            
        elif layer['type'] == 'MaxPooling':
            model.add(MaxPooling2D(pool_size=(2, 2), 
                                   strides=layer['strides'], 
                                   padding='same'))
                                   
        elif layer['type'] == 'Flatten':
            model.add(Flatten())
            
        elif layer['type'] == 'Dense':
            model.add(Dense(units=layer['units'], 
                            activation=layer['activation']))
                            
        elif layer['type'] == 'Dropout':
            model.add(Dropout(rate=layer['rate']))
            
        elif layer['type'] == 'DontCare':
            # Añadir la capa 'DontCareLayer' como parte del modelo
            model.add(DontCareLayer())
    
    # Construir el modelo especificando el input_shape
    model.build(input_shape=(None,) + input_shape)  # Ajusta esto al tamaño de entrada

    return model



Layer: {'type': 'Flatten'} -> Binary Encoding: 10100000
Layer: {'type': 'Conv2D', 'filters': 32, 'strides': 1, 'activation': 'relu'} -> Binary Encoding: 00000000
Layer: {'type': 'BatchNorm'} -> Binary Encoding: 00100000
Layer: {'type': 'MaxPooling', 'strides': 2} -> Binary Encoding: 01010000
Layer: {'type': 'Flatten'} -> Binary Encoding: 10100000
Layer: {'type': 'Dense', 'units': 256, 'activation': 'relu'} -> Binary Encoding: 10000000
Layer: {'type': 'Dropout', 'rate': 0.5} -> Binary Encoding: 01111000
Layer: {'type': 'Dense', 'units': 1, 'activation': 'sigmoid'} -> Binary Encoding: 10011100
Total Alleles: 64
Total Alleles (after padding): 784
Final Binary Encoding: 1010000000000000001000000101000010100000100000000111100010011100111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100

In [110]:
# Ejemplo 1: build_CNN_LF_model
model_CNN_LF = {
    "layers": [
        {"type": "Conv2D", "filters": 30, "strides": 1, "activation": "relu"},
        {"type": "Dropout", "rate": 0.2},
        {"type": "BatchNorm"},
        {"type": "MaxPooling", "strides": 2},
        {"type": "Conv2D", "filters": 16, "strides": 1, "activation": "relu"},  # Revisar 'filters'
        {"type": "Dropout", "rate": 0.2},
        {"type": "BatchNorm"},
        {"type": "MaxPooling", "strides": 2},
        {"type": "Flatten"},
        {"type": "Dense", "units": 256, "activation": "relu"},
        {"type": "Dropout", "rate": 0.3},
        {"type": "Dense", "units": 1, "activation": "sigmoid"}  # Revisar 'units'
    ]
}

# Ejemplo 2: build_reduced_model
model_reduced = {
    "layers": [
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 16, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 8, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Flatten"},
        {"type": "Dense", "units": 32, "activation": "leaky_relu"},
        {"type": "Dense", "units": 1, "activation": "sigmoid"}  # Revisar 'units'
    ]
}

# Ejemplo 3: build_Spectro_CNN_model
model_spectro_CNN = {
    "layers": [
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "MaxPooling", "strides": 2}, 
         
         
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
         {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
         {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
    {"type":"Flatten"},
    {"type":"Dense","units":256,"activation":"relu"},   
    {"type":"Dropout","rate":0.5},
                
        {"type": "Dense", "units": 1, "activation": "sigmoid"}  # Revisar 'units'
    ]
}

# Ejemplo 4: Simple Conv2D model
model_simple_conv = {
    "layers": [
        {"type": "Conv2D", "filters": 16, "strides": 1, "activation": "relu"},
        {"type": "Flatten"},
        {"type": "Dense", "units": 128, "activation": "relu"},
        {"type": "Dense", "units": 10, "activation": "sigmoid"}  # Revisar 'units'
    ]
}

# Ejemplo 5: Simple Dense model
model_dense_only = {
    "layers": [
        {"type": "Flatten"},
        {"type": "Dense", "units": 256, "activation": "relu"},
        {"type": "Dense", "units": 128, "activation": "relu"},
        {"type": "Dense", "units": 256, "activation": "sigmoid"}
    ]
}

# Ejemplo 6: Small CNN model with Dropout
model_small_CNN = {
    "layers": [
        {"type": "Conv2D", "filters": 8, "strides": 1, "activation": "relu"},
        {"type": "Dropout", "rate": 0.2},
        {"type": "MaxPooling", "strides": 2},
        {"type": "Flatten"},
        {"type": "Dense", "units": 32, "activation": "relu"},
        {"type": "Dense", "units": 1, "activation": "sigmoid"}  # Revisar 'units'
    ]
}

# Ejemplo 7: Deep CNN model
model_deep_CNN = {
    "layers": [
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "relu"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "relu"},
        {"type": "MaxPooling", "strides": 2},
        {"type": "Flatten"},
        {"type": "Dense", "units": 32, "activation": "relu"},
        {"type": "Dense", "units": 10, "activation": "sigmoid"}  # Revisar 'units'
    ]
}

# Ejemplo 8: Basic Dense with Dropout
model_dense_dropout = {
    "layers": [
        {"type": "Flatten"},
        {"type": "Dense", "units": 128, "activation": "relu"},
        {"type": "Dropout", "rate": 0.5},
        {"type": "Dense", "units": 32, "activation": "tanh"}
    ]
}

model_with_depthwise = {
    "layers": [
        {"type": "DepthwiseConv2D", "filters": 16, "strides": 1, "activation": "relu"},
        {"type": "BatchNorm"},
        {"type": "MaxPooling", "strides": 2},
        {"type": "Dense", "units": 128, "activation": "sigmoid"},
        {"type": "Dense", "units": 1, "activation": "tanh"}
    ]
}
model_with_global_avg_pooling = {
    "layers": [
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "relu"},
        {"type": "BatchNorm"},
        {"type": "GlobalAveragePooling2D"},
        {"type": "Dense", "units": 32, "activation": "relu"},
        {"type": "Dense", "units": 1, "activation": "sigmoid"}
    ]
}


In [111]:
# Función para validar si los valores del modelo son válidos
def validate_model_layers(model):
    for layer in model['layers']:
        layer_type = layer['type']
        
        if layer_type == 'Conv2D':
            filters = layer.get('filters')
            strides = layer.get('strides')
            activation = layer.get('activation')

            if filters not in filter_options:
                print(f"Invalid filters value in Conv2D layer: {filters}")
                return False
            if strides not in stride_options:
                print(f"Invalid strides value in Conv2D layer: {strides}")
                return False
            if activation not in activation_options:
                print(f"Invalid activation value in Conv2D layer: {activation}")
                return False
        
        elif layer_type == 'Dense':
            units = layer.get('units')
            activation = layer.get('activation')

            if units not in neuron_options:
                print(f"Invalid units value in Dense layer: {units}")
                return False
            if activation not in activation_options:
                print(f"Invalid activation value in Dense layer: {activation}")
                return False

        elif layer_type == 'Dropout':
            rate = layer.get('rate')
            if rate not in dropout_options:
                print(f"Invalid dropout rate value: {rate}")
                return False
        
        elif layer_type == 'MaxPooling':
            strides = layer.get('strides')
            if strides not in stride_options:
                print(f"Invalid strides value in MaxPooling layer: {strides}")
                return False
        
        elif layer_type == 'Flatten':
            # No se necesitan validaciones adicionales para Flatten
            pass

        elif layer_type == 'BatchNorm':
            # No se necesitan validaciones adicionales para BatchNorm
            pass

        elif layer_type == 'DepthwiseConv2D':
            filters = layer.get('filters')
            strides = layer.get('strides')
            activation = layer.get('activation')

            if filters not in filter_options:
                print(f"Invalid filters value in DepthwiseConv2D layer: {filters}")
                return False
            if strides not in stride_options:
                print(f"Invalid strides value in DepthwiseConv2D layer: {strides}")
                return False
            if activation not in activation_options:
                print(f"Invalid activation value in DepthwiseConv2D layer: {activation}")
                return False

        elif layer_type == 'DontCare':
            # La capa 'DontCare' no tiene parámetros, por lo tanto no requiere validación adicional
            pass
        
        layer_type = layer['type']
        if layer_type not in layer_type_options:
            print(f"Invalid layer type: {layer_type}")
            return False
        
    return True


# Función para verificar y construir el modelo
def run_verification_and_build(model):
    if validate_model_layers(model):
        print("\nModel is valid. Running verification and building the model:")
        # Verificación del modelo
        if verify_model_architecture(model):
            print("Model verification passed.")
        else:
            print("Model verification failed.")
        
        # Construcción del modelo en TensorFlow desde el diccionario decodificado
        decoded_model_dict = decode_model_architecture(encode_model_architecture(model))
        tf_model = build_tf_model_from_dict(decoded_model_dict)

        # Mostrar la estructura del modelo de TensorFlow
        try:
            tf_model.summary()
        except Exception as e:
            print(f"Error building model: {e}")
    else:
        print("Model is invalid. Skipping verification and build.")
    print("\n")



In [112]:

# Verificación de los modelos
print("Verifications:")

print("\nModel 1: CNN_LF")
run_verification_and_build(model_CNN_LF)  # Ejemplo 1

print("\nModel 2: Reduced")
run_verification_and_build(model_reduced)  # Ejemplo 2

print("\nModel 3: Spectro CNN")
run_verification_and_build(model_spectro_CNN)  # Ejemplo 3

print("\nModel 4: Simple Conv2D")
run_verification_and_build(model_simple_conv)  # Ejemplo 4

print("\nModel 5: Dense Only")
run_verification_and_build(model_dense_only)  # Ejemplo 5

print("\nModel 6: Small CNN")
run_verification_and_build(model_small_CNN)  # Ejemplo 6

print("\nModel 7: Deep CNN")
run_verification_and_build(model_deep_CNN)  # Ejemplo 7

print("\nModel 8: Dense with Dropout")
run_verification_and_build(model_dense_dropout)  # Ejemplo 8

print("\nModel with DepthwiseConv2D")
run_verification_and_build(model_with_depthwise)

print("\nModel with GlobalAveragePooling2D")
run_verification_and_build(model_with_global_avg_pooling)

Verifications:

Model 1: CNN_LF

Model is valid. Running verification and building the model:
Layer: {'type': 'Conv2D', 'filters': 30, 'strides': 1, 'activation': 'relu'} -> Binary Encoding: 00001000
Layer: {'type': 'Dropout', 'rate': 0.2} -> Binary Encoding: 01100000
Layer: {'type': 'BatchNorm'} -> Binary Encoding: 00100000
Layer: {'type': 'MaxPooling', 'strides': 2} -> Binary Encoding: 01010000
Layer: {'type': 'Conv2D', 'filters': 16, 'strides': 1, 'activation': 'relu'} -> Binary Encoding: 00010000
Layer: {'type': 'Dropout', 'rate': 0.2} -> Binary Encoding: 01100000
Layer: {'type': 'BatchNorm'} -> Binary Encoding: 00100000
Layer: {'type': 'MaxPooling', 'strides': 2} -> Binary Encoding: 01010000
Layer: {'type': 'Flatten'} -> Binary Encoding: 10100000
Layer: {'type': 'Dense', 'units': 256, 'activation': 'relu'} -> Binary Encoding: 10000000
Layer: {'type': 'Dropout', 'rate': 0.3} -> Binary Encoding: 01101000
Layer: {'type': 'Dense', 'units': 1, 'activation': 'sigmoid'} -> Binary Encodin

# Testing random generated architectures


In [113]:
import random

# Función para generar una capa aleatoria en binario
def generate_random_layer():
    # Seleccionamos aleatoriamente un tipo de capa
    layer_type = random.choice(list(layer_type_options.keys()))

    if layer_type == 'Conv2D':
        filters = random.choice(list(filter_options.keys()))
        strides = random.choice(list(stride_options.keys()))
        activation = random.choice(list(activation_options.keys()))
        binary_representation = (
            layer_type_options[layer_type] + 
            filter_options[filters] + 
            stride_options[strides] + 
            activation_options[activation]
        )
    elif layer_type == 'Dense':
        neurons = random.choice(list(neuron_options.keys()))
        activation = random.choice(list(activation_options.keys()))
        binary_representation = (
            layer_type_options[layer_type] + 
            neuron_options[neurons] + 
            activation_options[activation]
        )
    elif layer_type == 'Dropout':
        dropout_rate = random.choice(list(dropout_options.keys()))
        binary_representation = (
            layer_type_options[layer_type] + 
            dropout_options[dropout_rate]
        )
    elif layer_type == 'MaxPooling':
        strides = random.choice(list(stride_options.keys()))
        binary_representation = layer_type_options[layer_type] + stride_options[strides]
    elif layer_type == 'Flatten':
        binary_representation = layer_type_options[layer_type]
    elif layer_type == 'BatchNorm':
        binary_representation = layer_type_options[layer_type]
    elif layer_type == 'DepthwiseConv2D':
        filters = random.choice(list(filter_options.keys()))
        strides = random.choice(list(stride_options.keys()))
        activation = random.choice(list(activation_options.keys()))
        binary_representation = (
            layer_type_options[layer_type] + 
            filter_options[filters] + 
            stride_options[strides] + 
            activation_options[activation]
        )
    elif layer_type == 'DontCare':
        binary_representation = layer_type_options[layer_type] + '00000'  # Añadimos bits extra

    # Rellenar hasta que cada capa tenga 8 bits si es necesario
    return binary_representation.ljust(8, '0')

# Función para generar una arquitectura completa en binario
def generate_random_model_binary(num_layers=5):
    # Generamos el número deseado de capas aleatorias
    architecture = []
    for _ in range(num_layers):
        layer_bin = generate_random_layer()
        architecture.append(layer_bin)
    
    # Concatenamos todas las capas en un solo string binario
    full_model_bin = ''.join(architecture)
    
    # Rellenamos hasta 600 alelos con capas Don't Care si es necesario
    while len(full_model_bin) < 600:
        full_model_bin += '11100000'  # '111' es la capa DontCare, el resto son ceros de relleno

    return full_model_bin[:600]  # Garantizamos que no exceda los 600 alelos


In [114]:
# Función para generar y mostrar 5 arquitecturas aleatorias y sus modelos de TensorFlow
def generate_and_print_tf_models(num_models=5):
    generated_tf_models = []
    
    for i in range(num_models):
        # Generar una arquitectura aleatoria con un número aleatorio de capas
        num_layers = random.randint(1, 75)  # Variamos el número de capas entre 1 y 75
        random_model_binary = generate_random_model_binary(num_layers=num_layers)
        
        # Decodificar el modelo generado
        decoded_model = decode_model_architecture(random_model_binary)
        print(f"Generated Model {i+1}:")
        print(decoded_model)
        # Validar el modelo decodificado
        is_valid_model = validate_model_layers(decoded_model)
        print(f"Model {i+1} is valid: {is_valid_model}")
        if is_valid_model:
            # Construir el modelo de TensorFlow usando el diccionario decodificado
            tf_model = build_tf_model_from_dict(decoded_model)
            generated_tf_models.append(tf_model)
            
            # Imprimir el modelo en formato summary de TensorFlow
            print(f"Model {i+1}:")
            tf_model.summary()
            print("\n" + "="*80 + "\n")
    
    return generated_tf_models

# Llamada a la función para generar e imprimir 5 modelos de TensorFlow
generated_tf_models = generate_and_print_tf_models(num_models=50)


Inserting Flatten before Dense layer.
Invalid placement: Replacing MaxPooling with DontCare after Flatten.
Invalid placement: Replacing MaxPooling with DontCare after Flatten.
Invalid placement: Replacing Conv2D with DontCare after Flatten.
Invalid placement: Replacing MaxPooling with DontCare after Flatten.
Invalid placement: Replacing DepthwiseConv2D with DontCare after Flatten.
Invalid placement: Replacing Conv2D with DontCare after Flatten.
Invalid placement: Replacing DepthwiseConv2D with DontCare after Flatten.
Invalid placement: Replacing MaxPooling with DontCare after Flatten.
Invalid placement: Replacing MaxPooling with DontCare after Flatten.
Invalid placement: Replacing MaxPooling with DontCare after Flatten.
Invalid placement: Replacing DepthwiseConv2D with DontCare after Flatten.
Invalid placement: Replacing DepthwiseConv2D with DontCare after Flatten.
Invalid placement: Replacing DepthwiseConv2D with DontCare after Flatten.
Invalid placement: Replacing Conv2D with DontCar

In [115]:
import random

# Función para generar mutaciones en una arquitectura conocida
def mutate_architecture(binary_model, mutation_rate=0.1):
    mutated_model = list(binary_model)  # Convertimos el string a lista para mutarlo
    for i in range(len(mutated_model)):
        if random.random() < mutation_rate:  # Decidir si mutar este bit
            # Mutar el bit (0 -> 1, 1 -> 0)
            mutated_model[i] = '1' if mutated_model[i] == '0' else '0'
    return ''.join(mutated_model)

# Función para realizar crossover entre dos modelos
def crossover(parent1, parent2):
    # Elegir un punto de corte al azar
    crossover_point = random.randint(1, len(parent1) - 1)
    
    # Generar hijos mezclando los padres
    child1 = parent1[:crossover_point] + parent2[crossover_point:]
    child2 = parent2[:crossover_point] + parent1[crossover_point:]
    
    return child1, child2

# Función para generar la población
def generate_population(known_architectures, population_size=100, mutation_rate=0.1):
    population = []

    # Paso 1: Añadir las arquitecturas conocidas a la población
    population.extend(known_architectures)

    # Paso 2: Generar el resto de la población
    while len(population) < population_size:
        # Seleccionar dos padres al azar de la población actual
        parent1 = random.choice(population)
        parent2 = random.choice(population)
        
        # Realizar crossover para generar nuevos hijos
        child1, child2 = crossover(parent1, parent2)
        
        # Aplicar mutaciones a los hijos
        child1 = mutate_architecture(child1, mutation_rate)
        child2 = mutate_architecture(child2, mutation_rate)
        
        # Añadir los hijos a la población
        population.append(child1)
        population.append(child2)

    # Asegurarse de que la población tiene el tamaño exacto
    return population[:population_size]

# Ejemplo de arquitecturas conocidas (en binario)
known_architectures = [
    '0000100001100000001000000101000000010000011000000010000001010000101000001000000001101000100111001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000',  # Ejemplo 1
    '0010000000010001001000000001100100100000101000001001001010011100111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000',  # Ejemplo 2
    '0000000100100000000000010101000000000001001000000000000100000001001000000000000100000001001000000000000100000001001000000000000100000001001000000000000100000001001000000000000100000001001000000000000100000001001000000000000100000001001000000000000100000001001000000000000100000001001000000000000100000001001000000000000100000001001000000000000100000001001000000000000100000001001000000000000100000001001000000000000100000001001000000000000100000001001000000000000100000001001000000000000100000001001000000000000100000001001000000000000100000001001000000000000100000001001000000000000100000001001000000000000100000001001000000000000100000001001000000000000100000001001000000000000100000001001000000000000100000001001000000000000100000001001000000000000110100000100000000111100010011100'  # Ejemplo 3
]

# Generar una población de 100 individuos
population = generate_population(known_architectures, population_size=100)

# Imprimir algunos individuos generados
for i, individual in enumerate(population):
    print(f"Individuo {i+1}: {individual}")
    decoded_chromosome = decode_model_architecture(individual)  # Decodificar y mostrar la arquitectura
    print(decoded_chromosome)


Individuo 1: 0000100001100000001000000101000000010000011000000010000001010000101000001000000001101000100111001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000
{'layers': [{'type': 'Conv2D', 'filters': 30, 'kernel_size': (3, 3), 'strides': 1, 'padding': 'same', 'activation': 'relu'}, {'type': 'Dropout', 'rate': 0.2}, {'type': 'BatchNorm'}, {'type': 'MaxPooling